In [ ]:
import pandas as pd

In [7]:
data = pd.read_excel("superstore.xls")


In [10]:
pd.set_option("display.max_columns", None)

In [11]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9657 entries, 0 to 9656
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9657 non-null   int64         
 1   Order ID       9657 non-null   object        
 2   Order Date     9657 non-null   datetime64[ns]
 3   Ship Date      9657 non-null   datetime64[ns]
 4   Ship Mode      9657 non-null   object        
 5   Customer ID    9657 non-null   object        
 6   Customer Name  9657 non-null   object        
 7   Segment        9657 non-null   object        
 8   Country        9657 non-null   object        
 9   City           9657 non-null   object        
 10  State          9657 non-null   object        
 11  Postal Code    9657 non-null   int64         
 12  Region         9657 non-null   object        
 13  Product ID     9657 non-null   object        
 14  Category       9657 non-null   object        
 15  Sub-Category   9657 n

In [28]:
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [31]:
conflicts = data.groupby('Product ID')['Product Name'].nunique()
conflicts = conflicts[conflicts > 1]
print(conflicts)


Series([], Name: Product Name, dtype: int64)


In [30]:
data = data[~data['Product ID'].isin(conflicts.index)]


In [6]:
# Convert to datetime in pandas before upload
data["Order Date"] = pd.to_datetime(data["Order Date"])
data["Ship Date"] = pd.to_datetime(data["Ship Date"])


In [8]:
data.to_csv("cleaned_data.csv",index=False)

In [9]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration
username = os.getenv("PG_USER")
password = os.getenv("PG_PASS")
host = 'localhost'
port = '5432'
database = 'Superstore'

# Check if credentials exist
if not username or not password:
    raise EnvironmentError("Missing USERNAME or MY_PASS environment variables")

# Connection string
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# Try connection
try:
    with engine.connect() as conn:
        print("Connected")
except Exception as e:
    print(f"Connection failed: {e}")



Connected


In [10]:
import pandas as pd
data = pd.read_csv("cleaned_data.csv")

In [11]:
data.to_sql("superstore_cleaned", engine, if_exists="replace",index=False)

657